In [1]:
import json
from azure.eventhub import EventHubProducerClient, EventData
from dotenv import load_dotenv
import os
import socket
import random

from random import randrange

load_dotenv()
EVENT_HUB_CONN_STRING = os.getenv("EVENT_HUB_CONN_STRING") 
EVENT_HUB_NAME_IMPRESSIONS = os.getenv("EVENT_HUB_NAME_IMPRESSIONS")
EVENT_HUB_NAME_CLICKS = os.getenv("EVENT_HUB_NAME_CLICKS")

eventHubConnString = EVENT_HUB_CONN_STRING
eventHubNameImpressions = EVENT_HUB_NAME_IMPRESSIONS
eventHubNameClicks = EVENT_HUB_NAME_CLICKS

producer_impressions = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameImpressions)
producer_clicks = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameClicks)

hostname = socket.gethostname()

In [2]:
from utils import generateClickEvent, generateImpressionEvent

def sendToEventsHub(jsonEvent, producer):
    eventString = json.dumps(jsonEvent)
    print(eventString) 
    event_data_batch = producer.create_batch() 
    event_data_batch.add(EventData(eventString)) 
    producer.send_batch(event_data_batch)

def generateEvents(isUnsupportedBrowser = False, isAnomaly = False):
    try:
        while True:
            impressionEvent = generateImpressionEvent(isUnsupportedBrowser, isAnomaly, None)    
            sendToEventsHub(impressionEvent, producer_impressions)
            if random.randint(1, 100) > 80:
                clickEvent = generateClickEvent(impressionEvent)    
                sendToEventsHub(clickEvent, producer_clicks)
    except KeyboardInterrupt:
        producer_impressions.close()
        producer_clicks.close()


In [3]:
generateEvents(False, False)

{"impressionId": "da2f9b9d-d62d-4194-a334-cb714649541b", "impressionDate": "2024-01-12T00:31:22", "productId": 910, "browser": "Safari", "browserVersion": "10.2", "device": "tablet", "source": "facebook", "ip_address": "8.5.2.153", "landing_page": "http://bennett.com/categoryindex.html", "page_loading_seconds": 4.611}
{"clickId": "ba773633-ae69-43bb-8982-78cefe5b917d", "impressionId": "da2f9b9d-d62d-4194-a334-cb714649541b", "clickDate": "2024-01-12T00:31:22", "productId": 910}
{"impressionId": "fc0a116a-642c-4c3d-b370-230f2d38b5db", "impressionDate": "2024-01-11T17:17:59", "productId": 714, "browser": "Firefox", "browserVersion": "8.5", "device": "computer", "source": "google", "ip_address": "21.197.133.196", "landing_page": "http://www.meyers.org/categories/tags/explorecategory.jsp", "page_loading_seconds": 6.453}
{"impressionId": "644dcc7f-6ec3-4f97-8eae-ad0915204c43", "impressionDate": "2024-01-10T18:30:44", "productId": 715, "browser": "Chrome", "browserVersion": "14.6", "device": 